Introduction to Transformers
============================

Welcome to the first notebook in the “Transformers Explained” series!
In this notebook, we will explore the basics of the Transformer
architecture, a model that has revolutionized natural language
processing (NLP) and other domains.

What is a Transformer?
-----------------------

The Transformer is a neural network architecture introduced in 2017 by
Vaswani et al. in the paper “Attention Is All You Need.” Unlike
previous models like RNNs and CNNs, the Transformer relies entirely on
attention mechanisms, removing the need for recurrence or convolutions.

![Transformer Architecture](https://jalammar.github.io/images/t/transformer_architecture.png)
*Figure: General Transformer Architecture (Source: Jalammar)*

Why Are Transformers Important?
-------------------------------

Before Transformers, sequential models like LSTM and GRU were dominant
in NLP. But they struggled with long-range dependencies and were hard to parallelize.
Transformers fixed that by using **self-attention**, enabling simultaneous processing.

General Architecture
--------------------

A typical Transformer consists of:
1.  **Encoder**: Processes the input and builds a contextual representation.
2.  **Decoder**: Uses the encoder output to generate the final sequence.

![Encoder Decoder](https://jalammar.github.io/images/t/transformer_encoder_decoder.png)
*Figure: Encoder and Decoder structure*

Encoder Blocks
--------------

Each encoder block has:
-   **Multi-head Self-Attention**
-   **Feed-Forward Neural Network**

![Self-Attention](https://jalammar.github.io/images/t/self-attention.png)
*Figure: Scaled Dot-Product Self-Attention*

Decoder Blocks
--------------

Each decoder block contains:
-   **Masked Multi-head Self-Attention**
-   **Encoder-Decoder Attention**
-   **Feed-Forward Layer**

![Multi-Head Attention](https://jalammar.github.io/images/t/multi-head-attention.png)
*Figure: Multi-head Attention explained*

Positional Encoding
--------------------

Transformers don’t use recurrence or convolutions, so they lack inherent word order.
To solve this, we add **Positional Encodings**.

![Positional Encoding](https://jalammar.github.io/images/t/transformer_positional_encoding.png)
*Figure: Positional encoding visualized as sinusoidal functions*

Simplified Example (Pseudo-code)
--------------------------------


In [ ]:
# Code remains unchanged
...

Conclusion
----------

This notebook introduced you to the basic concepts behind the Transformer architecture.
In the next notebooks, we’ll dive deeper into the **attention mechanism**, and build
a full Transformer step-by-step.
